In [1]:
import re
import pandas as pd
import numpy as np

In [7]:
csv = pd.read_csv('parsing_test_data.csv', names=['Col'])

In [8]:
csv.head(50)

,Col
0,UNIQUE-ID - RXN-12314
1,TYPES - Chemical-Reactions
2,TYPES - Small-Molecule-Reactions
3,ATOM-MAPPINGS - (:NO-HYDROGEN-ENCODING (0 1 2 ...
4,CITATIONS - 17399751
5,COMMENT - A heme-thiolate protein (P-450). Clo...
6,CREDITS - SRI
7,CREDITS - caspi
8,"DBLINKS - (RHEA ""33012"" NIL |kothari| 37093103..."
9,ENZYMATIC-REACTION - ENZRXN-19061


In [50]:
k1 = 0
ID = [] # 'UNIQUE-ID - '
ID_L = len('UNIQUE-ID - ')

k2 = 0
Erxn = [] # Enzymatic Reaction - 
Erxn_L = len('ENZYMATIC-REACTION - ')

k3 = 0
gibbs = [] #Gibbs Free energy - 
gibbs_L = len('GIBBS-0 - ')

k6 = 0
ecnum = [] #Gibbs Free energy - 
ecnum_L = len('EC-NUMBER - ')

k4 = 0
prod = [] # PRODUCTS - 
prod_L = len('RIGHT - ')

k5 = 0
sub = [] # SUBSTRATES - 
sub_L = len('LEFT - ')

substrate = []
product = []
reaction = []
for i in range(csv.shape[0]):
    
    data = csv['Col'][i]
    if data[:ID_L] == 'UNIQUE-ID - ':
        ID.append(data[(ID_L):])
        k1 += 1
        #print(data[(ID_L):])
    
    elif data[:gibbs_L] == 'GIBBS-0 - ':
        gibbs.append(data[(gibbs_L):])
        k3 += 1
    elif data[:ecnum_L] == 'EC-NUMBER - ':
        ecnum.append(data[(ecnum_L):])
        k6 += 1    
    
    elif data[:prod_L] == 'RIGHT - ':
        product.append(data[(prod_L):])
        # k4 += 1
    elif data[:sub_L] == 'LEFT - ':
        substrate.append(data[(sub_L):]) # there are several substrate which will be appended to another list first
        # k5 += 1
    elif data[:Erxn_L] == 'ENZYMATIC-REACTION - ':
        reaction.append(data[(Erxn_L):])
    else:
        pass
    
    if k3 < k1-1:
        gibbs.append('N/A')
        k3 += 1
    if k6 < k1-1:
        ecnum.append('N/A')
        k6 += 1
    if k4 < k1-1:
        if product != []:
            prod.append(product)
            product = []
        else:
            prod.append('N/A')
        k4 += 1
    if k5 < k1-1:
        #print(substrate)
        if substrate != []:
            sub.append(substrate)
            substrate = []
        else:
            sub.append('N/A')
        k5 += 1
        
    if k2 < k1-1:
        #print(substrate)
        if reaction != []:
            Erxn.append(reaction)
            reaction = []
        else:
            Erxn.append('N/A')
        k2 += 1

# double check row count
if k4 < k1:
    if product != []:
        prod.append(product)
        product = []
    else:
        prod.append('N/A')
    k4 += 1
if k5 < k1:
    #print(substrate)
    if substrate != []:
        sub.append(substrate)
        substrate = []
    else:
        sub.append('N/A')
    k5 += 1
    
if k2 < k1:
    #print(substrate)
    if reaction != []:
        Erxn.append(reaction)
        reaction = []
    else:
        Erxn.append('N/A')
    k2 += 1

print ('Number of compounds count is', k1, k2, k3, 'rxn', k4, 'and alt-subs', k5)

Number of compounds count is 10 10 10 rxn 10 and alt-subs 10


In [51]:
listednames = [ID, gibbs, ecnum, prod, sub, Erxn]
for item in listednames:
    print(len(item))


10
10
10
10
10
10


In [52]:
df_rxns = pd.DataFrame({'UNIQUE-ID' : ID,
                    'ERXN-NUMBER': Erxn,
                    'EC-NUMBER': ecnum,
                    'SUBSTRATES' : sub,
                    'PRODUCTS' : prod,
                    'GIBBS' :gibbs})

In [53]:
df_rxns

,UNIQUE-ID,ERXN-NUMBER,EC-NUMBER,SUBSTRATES,PRODUCTS,GIBBS
0,RXN-12314,[ENZRXN-19061],N/A,"[CPD-7557, Red-NADPH-Hemoprotein-Reductases, O...","[CPD-13248, Ox-NADPH-Hemoprotein-Reductases, W...",-108.40886
1,RXN-16877,[ENZRXN-24025],N/A,[Cys-Cys-HlmE],[4N-2mthio-5oxo-3Spyrrolidine-2-COOH-HlmE],-29.53003
2,PORPHOBILSYNTH-RXN,"[PORPHOBILSYNTH-ENZRXN, ENZRXN66-1464, ENZRXN-...",EC-4.2.1.24,[5-AMINO-LEVULINATE],"[PROTON, WATER, PORPHOBILINOGEN]",-33.708138
3,RXN-9204,[ENZRXN-14772],EC-2.5.1.74,"[CPD-21340, DIHYDROXYNAPHTHOATE, PROTON]","[CPD-12118, PPI, CARBON-DIOXIDE]",-16.385834
4,RXN-9516,"[ENZRXN-15226, ENZRXN3O-10296, ENZRXN1G-511, E...",EC-2.3.1.179,"[Butanoyl-ACPs, MALONYL-ACP, PROTON]","[3-oxo-hexanoyl-ACPs, CARBON-DIOXIDE, ACP]",-6.862854
5,RXN-20002,N/A,EC-2.3.1.41,"[ENTEROBACTIN, FE+3]","[FERRIC-ENTEROBACTIN-COMPLEX, PROTON]",-29.63234
6,RXN-14284,[ENZRXN0-7678],EC-2.4.1.1,"[MALTOPENTAOSE, Pi]","[MALTOTETRAOSE, GLC-1-P]",-10.329956
7,RXN-10660,[ENZRXN-16661],EC-4.2.1.59,[3-hydroxy-cis-D9-hexaecenoyl-ACPs],"[Trans-D3-cis-D9-hexadecenoyl-ACPs, WATER]",22.705242
8,RXN-12798,[ENZRXN-19789],N/A,"[Poly-Gamma-D-Glutamate, GLT, ATP]","[Poly-Gamma-D-Glutamate, ADP, Pi]",38.10034
9,2-FUROATE--COA-LIGASE-RXN,[ENZRXN-20049],EC-6.2.1.31,"[CO-A, 2-FUROATE, ATP]","[CPD-73, PPI, AMP]",-12.711792


### IGNORE THE REMAINDER OF THIS NOTEBOOK 
#### *cries internally*

In [ ]:
    'unique': re.compile('(UNIQUE.*)'), 1
    'product': re.compile('(RIGHT)' + '.*?' + '([\w.]+$)'), m
    'substrate': re.compile('(LEFT)' + '.*?' + '\\s+' + '$'), m
    'ec': re.compile('(EC.*)' + '(NUMBER)' + '.*?' + '([\w.]+$)'), 1
    'gibbs': re.compile('(GIBBS.*)' + '\d' + '\\s+') 1
        enzymatic reaction 1

rx_dict = {
    'unique': re.compile(r'(UNIQUE-ID = (?P<unique>))'),
    'product': re.compile('(RIGHT)' + '.*?' + '([\w.]+$)'),
    'substrate': re.compile('(LEFT)' + '.*?' + '\\s+' + '$'),
    'ec': re.compile('(EC.*)' + '(NUMBER)' + '.*?' + '([\w.]+$)'),
    'gibbs': re.compile('(GIBBS.*)' + '\d' + '\\s+')
}

In [2]:
rx_dict = {
    'unique': re.compile('(UNIQUE.*)'),
    'product': re.compile('(RIGHT)' + '.*?' + '([\w.]+$)'),
    'substrate': re.compile('(LEFT)' + '.*?' + '\\s+' + '$'),
    'ec': re.compile('(EC.*)' + '(NUMBER)' + '.*?' + '([\w.]+$)'),
    'gibbs': re.compile('(GIBBS.*)' + '\d' + '\\s+')
}

In [3]:
def _parse_line(line):
    """
    Do a regex search against all defined regexes and
    return the key and match result of the first matching regex

    """


    for key, rx in rx_dict.items():
        match = rx.search(line)
        

        if match:
            key, match
            row = {}

            if key == 'unique':
                unique = match.group(0)
                umatch.append(unique)
            elif key == 'gibbs':
                gibbs = match.group(0)
                gmatch.append(gibbs)
            elif key == 'substrate':
                substrate = match.group(0)
                smatch.append(substrate)
                
            elif key == 'product':
                product = match.group(0)
                pmatch.append(product)
            elif key == 'ec':
                ec = match.group(0)
                ematch.append(ec)
            
            row.update({
                    'unique': umatch,
                    'product': pmatch,
                    'substrate': smatch,
                    'ec': ematch,
                    'gibbs': gmatch,
            })
    # if there are no matches
    return row

def _parse_line(line):
    """
    Do a regex search against all defined regexes and
    return the key and match result of the first matching regex

    """
    global unique, gibbs, substrate, product, ec, row
    umatch = []
    gmatch = []
    smatch = []
    pmatch = []
    ematch = []    
    row = {}
    for key, rx in rx_dict.items():
        match = rx.search(line)
        if match:
            key, match

        if key == 'unique':
            unique = match.group(0)
            umatch.append(unique)
        if key == 'gibbs':
            gibbs = match.group(0)
            gmatch.append(gibbs)
        if key == 'substrate':
            substrate = match.group(0)
            smatch.append(substrate)
                
        if key == 'product':
            product = match.group(0)
            pmatch.append(product)
        if key == 'ec':
            ec = match.group(0)
            ematch.append(ec)
            
        row.update({
                    'unique': umatch,
                    'product': pmatch,
                    'substrate': smatch,
                    'ec': ematch,
                    'gibbs': gmatch,
            })
    # if there are no matches
    return row

In [28]:
lists = []
with open('parsing_test_data.txt') as infile:
    copy = False
    for line in infile:
        for key, value in rx_dict.items():
            if re.match(value, line): 
                copy = True
                _parse_line(line)
            elif line.startswith('//'):
                copy = False
            elif copy:
                rxn_list = []
                rxn_list.append(line)
            lists.append(rxn_list)

In [37]:
rx_dict.values()

dict_values([re.compile('(UNIQUE.*)'), re.compile('(RIGHT).*?([\\w.]+$)'), re.compile('(LEFT).*?\\s+$'), re.compile('(EC.*)(NUMBER).*?([\\w.]+$)'), re.compile('(GIBBS.*)\\d\\s+')])

In [86]:
with open('parsing_test_data.txt') as input_data:
    # Skips text before the beginning of the interesting block:
    for line in input_data:
        if line.strip() == 'UNIQUE':  # Or whatever test is needed
            break
    # Reads text until the end of the block:
    for line in input_data:  # This keeps reading the file
        if line.strip() == '//':
            break
        print(line)  # Line is extracted (or block_of_lines.append(line), etc.)

In [102]:
with open("test.txt") as f:
    for line in f:
        if line.rstrip() != "---a":
            print("")
            for line in f:
                if line.rstrip() == "---a":
                    break
                print(line.rstrip())


foo bar
lorem ipsum
dolor

bar bar
foobar
foob


with open("parsing_test_data.txt") as f:
    for line in f:
        if re.match('(UNIQUE.*)', line):
            print("", line.rstrip())
            for line in f:
                if line.rstrip() == "//":
                    break
                print(line.rstrip())
with open("parsing_test_data.txt") as f:
    for line in f:
        if re.match('(UNIQUE.*)', line):
            print(line.strip())
            for line in f:
                if line.rstrip() == "//":
                    break
                print(line.rstrip())

In [5]:
k = 0
m = 0
umatch = []
gmatch = []
smatch = []
pmatch = []
ematch = []    
with open("parsing_test_data.txt") as f:
    for line in f:
        if re.match('(UNIQUE.*)', line):
            row = _parse_line(line)
            for line in f:
                if line.rstrip() == "//":
                    break
                print(line.rstrip())
    print(k, m, row)

TYPES - Chemical-Reactions
TYPES - Small-Molecule-Reactions
"ATOM-MAPPINGS - (:NO-HYDROGEN-ENCODING (0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 33 27 28 29 30 31 32 34 35 36) (((""Ox-NADPH-Hemoprotein-Reductases"" 0 18) (""CPD-13248"" 19 35) (""WATER"" 36 36)) ((""Red-NADPH-Hemoprotein-Reductases"" 0 18) (""CPD-7557"" 19 34) (""OXYGEN-MOLECULE"" 35 36))))"
CITATIONS - 17399751
COMMENT - A heme-thiolate protein (P-450). Cloned from |FRAME: TAX-35608|. Part of the biosynthetic pathway of artemisinin.
CREDITS - SRI
CREDITS - caspi
"DBLINKS - (RHEA ""33012"" NIL |kothari| 3709310382 NIL NIL)"
ENZYMATIC-REACTION - ENZRXN-19061
GIBBS-0 - -108.40886
IN-PATHWAY - RXN-13432
IN-PATHWAY - PWY-5195
LEFT - CPD-7557
LEFT - Red-NADPH-Hemoprotein-Reductases
LEFT - OXYGEN-MOLECULE
ORPHAN? - :NO
PHYSIOLOGICALLY-RELEVANT? - T
REACTION-DIRECTION - PHYSIOL-LEFT-TO-RIGHT
RIGHT - CPD-13248
RIGHT - Ox-NADPH-Hemoprotein-Reductases
RIGHT - WATER
RIGHT - PROTON
TYPES - Protein-Modific

In [5]:
flip = False
k = 0
m = 0
umatch = []
gmatch = []
smatch = []
pmatch = []
ematch = []    
row = {}
with open('parsing_test_data.txt', 'r') as f:
    for line in f:
        for value in rx_dict.values():
            if re.match(value, line): 
                _parse_line(line)
                k+=1
                print(row)
            if line.startswith('//'): 
                m += 1
                pass
    print(k, m)

{'unique': ['UNIQUE-ID - RXN-12314'], 'product': [], 'substrate': [], 'ec': [], 'gibbs': []}
{'unique': ['UNIQUE-ID - RXN-12314'], 'product': [], 'substrate': [], 'ec': [], 'gibbs': ['GIBBS-0 - -108.40886    \n']}
{'unique': ['UNIQUE-ID - RXN-12314'], 'product': [], 'substrate': ['LEFT - CPD-7557\n'], 'ec': [], 'gibbs': ['GIBBS-0 - -108.40886    \n']}
{'unique': ['UNIQUE-ID - RXN-12314'], 'product': [], 'substrate': ['LEFT - CPD-7557\n', 'LEFT - Red-NADPH-Hemoprotein-Reductases\n'], 'ec': [], 'gibbs': ['GIBBS-0 - -108.40886    \n']}
{'unique': ['UNIQUE-ID - RXN-12314'], 'product': [], 'substrate': ['LEFT - CPD-7557\n', 'LEFT - Red-NADPH-Hemoprotein-Reductases\n', 'LEFT - OXYGEN-MOLECULE\n'], 'ec': [], 'gibbs': ['GIBBS-0 - -108.40886    \n']}
{'unique': ['UNIQUE-ID - RXN-12314'], 'product': ['RIGHT - CPD-13248'], 'substrate': ['LEFT - CPD-7557\n', 'LEFT - Red-NADPH-Hemoprotein-Reductases\n', 'LEFT - OXYGEN-MOLECULE\n'], 'ec': [], 'gibbs': ['GIBBS-0 - -108.40886    \n']}
{'unique': ['UNI

flip = False
k = 0
m = 0
with open('parsing_test_data.txt', 'r') as f:
    for line in f:
        for element in rx_dict:
            if re.search(element, line):
                print('hello', element)


In [65]:
row

{'unique': 'UNIQUE-ID - 2-FUROATE--COA-LIGASE-RXN',
 'product': 'RIGHT - AMP',
 'substrate': 'LEFT-TO-RIGHT\n',
 'ec': 'EC-NUMBER - EC-6.2.1.31',
 'gibbs': 'GIBBS-0 - -12.711792    \n',
 'hello': 'hello',
 'i': 5,
 'j': 1}

flip = False
with open('parsing_test_data.txt', 'r') as f:
    for line in f:
        if not flip and re.match('^[a-zA-Z0-9_]',line): flip = True
        if flip:
            _parse_line(line)
            if re.match('//',line): flip = False

In [60]:
def parse_file(filepath):
    """
    Parse text at given filepath

    Parameters
    ----------
    filepath : str
        Filepath for file_object to be parsed

    Returns
    -------
    data : pd.DataFrame
        Parsed data

    """

    data = []  # create an empty list to collect the data
    # open the file and read through it line by line
    flip = False
    with open('parsing_test_data.txt', 'r') as f:
        for line in f:
            if not flip and re.match('^[a-zA-Z0-9_]',line): flip = True
            if re.match('//',line): flip = False
            if flip:
                _parse_line(line)
                

                    # append the dictionary to the data list
                data.append(row)
                    


        # create a pandas DataFrame from the list of dicts
        df = pd.DataFrame(data)
        # set the School, Grade, and Student number as the index
        #data.set_index(['School', 'Grade', 'Student number'], inplace=True)
        # consolidate df to remove nans
        #data = data.groupby(level=data.index.names).first()
        # upgrade Score from float to integer
        #data = data.apply(pd.to_numeric, errors='ignore')
    return data

In [61]:
parse_file('parsing_test_data.txt')

UnboundLocalError: local variable 'product' referenced before assignment

In [44]:
pd.DataFrame(data)

""


 with open('parsing_test_data.txt', 'r') as f:
    for line in f:
        for key, rx in rx_dict.items():
            match = rx.search(line)
            if match:
                print(key, match)
            else:
                print('No matches')
                    

In [7]:
import tarfile
import numpy as np
import pandas as pd

tar = tarfile.open('../../datasets/meta.tar.gz', 'r:gz')
for member in tar.getmembers():
    f = tar.extractfile(member)
    if f is not None:
        content = f.read()

tf = tarfile.open("../../datasets/meta.tar.gz")
tf.extractall()

In [ ]:
dfembl = pd.read_csv('22.6/data/reactions.dat', skiprows=67, sep='(\/{2,3}/g)', header=None)

In [ ]:
dfembl

In [ ]:
id_list = []
for record in SeqIO.parse('22.6/data/reactions.xml', 'seqxml'):
    id_list.append(record.id)

In [ ]:
id_list

In [ ]:
from Bio import SeqIO
uniprot = SeqIO.index("22.6/data/reactions.dat", "embl")
#with open("selected.dat", "wb") as out_handle:
    #out_handle.write(reactions_test.embl)
dfembl = pd.DataFrame(eval(uniprot))

In [ ]:
len(uniprot)

In [ ]:
records = SeqIO.parse("22.6/data/reactions.dat", "embl")
count = SeqIO.write(records, "test_reactions.embl", "embl")
print('Converted %i records' %count)
#with open(uniprot, "wb") as out_handle:
 #   data.append(out_handle)

        #for row in result_set:
    #data.append({'value': row["tag_expression"], 'key': row["tag_name"]})

In [ ]:
ugghhh = pd.DataFrame(eval(uniprot))

In [ ]:
dfa = pd.read_csv('22.6/data/reactions.dat', skiprows=67, sep='(\/{2,3}/g)', header=None)

In [ ]:
dfa.head(30)

In [ ]:
biglist2 = dfa.values.tolist()
biglist2

In [ ]:
from itertools import groupby

def breakby(biglist2, sep, delim=None):
    for item in biglist2:
        p = item.split(sep)
        yield p[0]
        if len(p) > 1:
            yield delim
            yield p[1]

smallerclist = [list(g) for k,g in groupby(breakby(biglist2, '//', None),
                                          lambda x: x is not None) if k]


In [ ]:
breakby(biglist2, '//', delim=None)

In [ ]:
colnames = ['Unique-ID', 'Types', 'Common-Name', 'Basis-for-assignment', 'Citations', 'Enzyme', 'Physiologically-relevant', 'Reaction']
df = pd.read_csv('22.6/data/enzrxns.dat', skiprows=60, sep='(\/{2,3}/g)', names=colnames, header=None)

In [ ]:
df.head(20)

In [ ]:
biglist1 = df['Unique-ID'].values.tolist()
biglist1

In [ ]:
from itertools import groupby

def breakby(biglist1, sep, delim=None):
    for item in biglist1:
        p = item.split(sep)
        yield p[0]
        if len(p) > 1:
            yield delim
            yield p[1]

smallerlist = [list(g) for k,g in groupby(breakby(biglist1, '//', None),
                                          lambda x: x is not None) if k]


In [ ]:
breakby(biglist1, '//', delim=None)

In [ ]:
smallerlist

In [ ]:
df3 = pd.DataFrame(smallerlist)

In [ ]:
df3.head()

In [ ]:
len(smallerlist)

In [ ]:
for innerlist in smallerlist:
    innerlist.sort(reverse=True)

In [ ]:
smallerlist

In [ ]:
testlist = [['^SUBSTRATE - ISOPENICILLIN-N',
  '^SUBSTRATE - CPD-207',
  '^CITATIONS - 8344300',
  '^CITATIONS - 8344300',
  'VMAX - 8.2800e-2',
  'VMAX - 0.57600003',
  'UNIQUE-ID - ENZRXN-24241',
  'TYPES - Enzymatic-Reactions',
  'TEMPERATURE-OPT - 25',
  'SPECIFIC-ACTIVITY - 1.2300e-2',
  'REGULATED-BY - REG-15201',
  'REGULATED-BY - REG-15200',
  'REGULATED-BY - REG-15199',
  'REGULATED-BY - REG-15198',
  'REGULATED-BY - REG-15197',
  'REGULATED-BY - REG-15196',
  'REACTION - 2.3.1.164-RXN',
  'PHYSIOLOGICALLY-RELEVANT? - T',
  'PH-OPT - 8.0:8.2',
  'KM - 6',
  'KM - 23',
  'ENZYME - CPLX-9042',
  'COMMON-NAME - isopenicillin N acyltransferase',
  'CITATIONS - 8396910:EV-EXP-IDA-PURIFIED-PROTEIN-NH:3648991251:hartmut',
  'CITATIONS - 8384123:EV-EXP-IDA-PURIFIED-PROTEIN-HH:3648991251:hartmut',
  'CITATIONS - 8344300:EV-EXP-IGI-FUNC-COMPLEMENTATION:3648991251:hartmut',
  'CITATIONS - 8344300:EV-EXP-IDA-PURIFIED-PROTEIN-NH:3648991251:hartmut',
  'CITATIONS - 8224864:EV-EXP-IDA-PURIFIED-PROTEIN-HH:3648991251:hartmut',
  'CITATIONS - 2829713:EV-EXP-IDA-PURIFIED-PROTEIN-NH:3648991251:hartmut',
  'CITATIONS - 2555269:EV-EXP:3648991251:hartmut',
  'CITATIONS - 2129535:EV-EXP:3648991251:hartmut',
  'CITATIONS - 2120195:EV-EXP-IDA-UNPURIFIED-PROTEIN-HH:3648991251:hartmut',
  'CITATIONS - 2110531:EV-EXP-IDA-PURIFIED-PROTEIN-NH:3648993653:hartmut',
  'CITATIONS - 16321143',
  'CITATIONS - 1581347:EV-EXP-IMP-REACTION-BLOCKED:3648991251:hartmut',
  'CITATIONS - 12709055:EV-EXP-IDA-PURIFIED-PROTEIN-HH:3648991251:hartmut',
  'BASIS-FOR-ASSIGNMENT - :MANUAL',
  '',
  ''],
 ['UNIQUE-ID - ENZRXN66-4540',
  'TYPES - Enzymatic-Reactions',
  'REACTION - FORMYLTHFGLUSYNTH-RXN',
  'PHYSIOLOGICALLY-RELEVANT? - T',
  'PH-OPT - 9.4',
  'ENZYME - HS06237-MONOMER',
  'CREDITS - brito',
  'CREDITS - SRI',
  'COMMON-NAME - folylpolyglutamate synthase',
  'CITATIONS - 8662720:EV-EXP-IDA-PURIFIED-PROTEIN:3584986140:brito',
  'BASIS-FOR-ASSIGNMENT - :AUTOMATED-NAME-MATCH',
  '',
  '']]

In [ ]:
testlist

In [ ]:
for innerlist in testlist:
    for item in innerlist:
        print(item)

In [ ]:
for innerlist in testlist:
    for item in innerlist:
        print(item.split(None, 1))
        

In [ ]:
new_testlist = []

In [ ]:
pd.DataFrame(df1.str.split('NaN').tolist(), columns=['Unique-ID'])

In [ ]:
df6 = pd.DataFrame(smallerlist)

In [ ]:
df6.head()

In [ ]:
df2

In [ ]:
pd.DataFrame(df2.str.split('NaN').tolist(), columns=['Unique-ID'])